In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


#this script takes some edgeR table results and analize the expressed genes, try to understad which genes are differentialy expressed among the samples. 
#the analysis based on q_val
import sys
!{sys.executable} -m pip install sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
import math
import re
import ast
import itertools
import scipy
import os
import openpyxl
from scipy.stats import fisher_exact



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
# creting list of possible 0,1 comb in list of 4 

In [2]:
N=4
K=1

a=[np.reshape(np.array(i), (K, N)) for i in itertools.product([0, 1], repeat = K*N)]

combs_0_1= [list(a[i][0]) for i in range(len(a))]

In [3]:
human_mouse_ortho_1_1

NameError: name 'human_mouse_ortho_1_1' is not defined

# define_groups_of_genes: 

## Hagai PIC4-LF high-low groups of human vs mouse

In [34]:
# open the big metadatand leave only 1-1
human_mouse_ortho=pd.read_csv(r'input\1-1_human_mouse_metadata.csv',index_col=0)
human_mouse_ortho_1_1=human_mouse_ortho[human_mouse_ortho['Mouse homology type']=='ortholog_one2one']
human_mouse_ortho_1_1


human_mouse_ortho_1_1.columns

# create 1-1 dict mouse>human ens names 
human_mouse_ortho_1_1_dict=dict(zip(human_mouse_ortho_1_1['Mouse gene stable ID'],human_mouse_ortho_1_1.index))

In [35]:
human_mouse_ortho_1_1_dict

{'ENSMUSG00000064341': 'ENSG00000198888',
 'ENSMUSG00000064345': 'ENSG00000198763',
 'ENSMUSG00000064351': 'ENSG00000198804',
 'ENSMUSG00000064354': 'ENSG00000198712',
 'ENSMUSG00000064356': 'ENSG00000228253',
 'ENSMUSG00000064357': 'ENSG00000198899',
 'ENSMUSG00000064358': 'ENSG00000198938',
 'ENSMUSG00000064360': 'ENSG00000198840',
 'ENSMUSG00000065947': 'ENSG00000212907',
 'ENSMUSG00000064363': 'ENSG00000198886',
 'ENSMUSG00000064367': 'ENSG00000198786',
 'ENSMUSG00000064368': 'ENSG00000198695',
 'ENSMUSG00000064370': 'ENSG00000198727',
 'ENSMUSG00000065495': 'ENSG00000207782',
 'ENSMUSG00000065424': 'ENSG00000284447',
 'ENSMUSG00000088059': 'ENSG00000252213',
 'ENSMUSG00000065501': 'ENSG00000199043',
 'ENSMUSG00000065208': 'ENSG00000263776',
 'ENSMUSG00000065242': 'ENSG00000239190',
 'ENSMUSG00000065444': 'ENSG00000207808',
 'ENSMUSG00000065159': 'ENSG00000207099',
 'ENSMUSG00000088385': 'ENSG00000212368',
 'ENSMUSG00000065053': 'ENSG00000206605',
 'ENSMUSG00000080486': 'ENSG000002

In [36]:
# define the groups: 

study='Hagai'
species=['homo','mus']
if species==['homo','mus']:
    species_full_names=['Homo sapiens','Mus musculus']
treats=['LF4','PIC4']
    
    


df_groups=pd.read_csv(r'input\{}_{}_vs_{}_{}_and_{}_groups_results.csv'.format(study,species[0],species[1],treats[0],treats[1]),index_col=0)
df_groups

DE_genes_FC_values_df=pd.read_csv(r'input\DE_genes_human_mouse_pic.csv',index_col=1)

DE_genes=DE_genes_FC_values_df.index
non_DE_genes=[i for i in human_mouse_ortho_1_1.index if i not in DE_genes]

# stim groups - high and low
group_stim_high=df_groups[(df_groups[f'group_{treats[1]}']==f'{treats[1]}_high_strict')|(df_groups[f'group_{treats[1]}']==f'{treats[1]}_high_moderate')]
print(len(group_stim_high.index))
group_stim_high_and_DE=[i for i in group_stim_high.index if i in DE_genes_FC_values_df.index]
print(len(group_stim_high_and_DE))


group_stim_low=df_groups[(df_groups[f'group_{treats[1]}']==f'{treats[1]}_low_strict')|(df_groups[f'group_{treats[1]}']==f'{treats[1]}_low_moderate')]
print(len(group_stim_low.index))
group_stim_low_and_DE=[i for i in group_stim_low.index if i in DE_genes_FC_values_df.index]
print(len(group_stim_low_and_DE))


# unst groups - high and low
group_unst_high=df_groups[(df_groups[f'group_{treats[0]}']==f'{treats[0]}_high_strict')|(df_groups[f'group_{treats[0]}']==f'{treats[0]}_high_moderate')]
print(len(group_unst_high.index))
group_unst_high_and_DE=[i for i in group_unst_high.index if i in DE_genes_FC_values_df.index]
print(len(group_unst_high_and_DE))


group_unst_low=df_groups[(df_groups[f'group_{treats[0]}']==f'{treats[0]}_low_strict')|(df_groups[f'group_{treats[0]}']==f'{treats[0]}_low_moderate')]
print(len(group_unst_low.index))
group_unst_low_and_DE=[i for i in group_unst_low.index if i in DE_genes_FC_values_df.index]
print(len(group_unst_low_and_DE))


579
224
519
166
498
104
476
84


In [37]:
DE_genes_new_filtration=DE_genes_FC_values_df.index
len(DE_genes_new_filtration)

1473

In [38]:
groups_list={f'{study}_{species[0]}_{species[1]}_{treats[1]}_high':group_stim_high_and_DE,
             f'{study}_{species[0]}_{species[1]}_{treats[1]}_low':group_stim_low_and_DE,
             f'{study}_{species[0]}_{species[1]}_{treats[0]}_high':group_unst_high_and_DE,
             f'{study}_{species[0]}_{species[1]}_{treats[0]}_low':group_unst_low_and_DE}


In [40]:
groups_list

{'Hagai_homo_mus_PIC4_high': ['ENSG00000164949',
  'ENSG00000138802',
  'ENSG00000204209',
  'ENSG00000155313',
  'ENSG00000109320',
  'ENSG00000149115',
  'ENSG00000169905',
  'ENSG00000143337',
  'ENSG00000198060',
  'ENSG00000164136',
  'ENSG00000162298',
  'ENSG00000178927',
  'ENSG00000111224',
  'ENSG00000007384',
  'ENSG00000179021',
  'ENSG00000115325',
  'ENSG00000146232',
  'ENSG00000147454',
  'ENSG00000132669',
  'ENSG00000168795',
  'ENSG00000081019',
  'ENSG00000168071',
  'ENSG00000137193',
  'ENSG00000015475',
  'ENSG00000136732',
  'ENSG00000163932',
  'ENSG00000107485',
  'ENSG00000184811',
  'ENSG00000135604',
  'ENSG00000006652',
  'ENSG00000136854',
  'ENSG00000138496',
  'ENSG00000184545',
  'ENSG00000153975',
  'ENSG00000205593',
  'ENSG00000079691',
  'ENSG00000116977',
  'ENSG00000176994',
  'ENSG00000180448',
  'ENSG00000269855',
  'ENSG00000118946',
  'ENSG00000032219',
  'ENSG00000116990',
  'ENSG00000196843',
  'ENSG00000122644',
  'ENSG00000140968',
  'ENS

In [32]:
#res_dir_top=r'\chip_seq_analysis_res\{}_{}_vs_{}_{}_and_{}_groups_moderate'.format(study,species[0],species[1],treats[0],treats[1])
res_dir_top=r'chip_seq_analysis_res\{}_{}_vs_{}_{}_and_{}_groups_moderate'.format(study,species[0],species[1],treats[0],treats[1])

os.makedirs(res_dir_top,exist_ok=True)

In [41]:
human_mouse_ortho_1_1

,Gene name human,Mouse gene stable ID,Mouse homology type,%id. target Mouse gene identical to query gene,%id. query gene identical to target Mouse gene,dN with Mouse,dS with Mouse,dnds,p_identity,HUMAN_PPODv4_Jaccard_dollo_age-depth,...,human_prom1M_H3K27ac_PIC4peaks_intersect_03,human_prom1M_H3K27ac_UNSTpeaks_intersect_03,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K27ac_PIC4peaks_intersect_00,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_00,mouse_prom1M_H3K27ac_PIC4peaks_intersect_03,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_03,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05
ENSG00000198888,MT-ND1,ENSMUSG00000064341,ortholog_one2one,77.0440,77.0440,NaN,NaN,NaN,77.04400,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198763,MT-ND2,ENSMUSG00000064345,ortholog_one2one,57.0605,57.3913,NaN,NaN,NaN,57.22590,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198804,MT-CO1,ENSMUSG00000064351,ortholog_one2one,90.8382,90.6615,NaN,NaN,NaN,90.74985,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198712,MT-CO2,ENSMUSG00000064354,ortholog_one2one,71.3656,71.3656,NaN,NaN,NaN,71.36560,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000228253,MT-ATP8,ENSMUSG00000064356,ortholog_one2one,45.5882,46.2687,0.4871,0.8480,0.574410,45.92845,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000125740,FOSB,ENSMUSG00000003545,ortholog_one2one,95.8580,95.8580,0.0211,0.3819,0.055250,95.85800,4.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000139648,KRT71,ENSMUSG00000051879,ortholog_one2one,91.2046,91.0305,0.0425,0.6850,0.062044,91.11755,4.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000185614,INKA1,ENSMUSG00000042106,ortholog_one2one,79.0941,80.4965,0.1151,0.4406,0.261235,79.79530,3.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000198400,NTRK1,ENSMUSG00000028072,ortholog_one2one,86.1809,85.8573,0.0769,0.6452,0.119188,86.01910,6.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


#### on chipseq - H3K27ac and H3K4me3


In [42]:
peaks='chip_seq'
#promoters=['prom1000100','prom2000500']
promoters=['prom1M']
intersects=['00','03','05']

if peaks=='chip_seq':
    #histon=['H3K27ac','H3K4me3']
    histon=['H3K27ac']
    treats_in_chip_r_atac=['PIC4','UNST']
elif peaks=='atac':
    histon=['atac']
    treats_in_chip_r_atac=['0h','4h','8h']

In [43]:
human_mouse_ortho_1_1_chip_seq_cols=human_mouse_ortho_1_1[[i for i in human_mouse_ortho_1_1.columns if 'peaks_intersect' in i]]
human_mouse_ortho_1_1_chip_seq_cols=human_mouse_ortho_1_1_chip_seq_cols.dropna()
human_mouse_ortho_1_1_chip_seq_cols

,human_prom1000100_H3K27ac_PIC4peaks_intersect_00,human_prom1000100_H3K27ac_UNSTpeaks_intersect_00,human_prom1000100_H3K27ac_PIC4peaks_intersect_03,human_prom1000100_H3K27ac_UNSTpeaks_intersect_03,human_prom1000100_H3K27ac_PIC4peaks_intersect_05,human_prom1000100_H3K27ac_UNSTpeaks_intersect_05,human_prom1000100_H3K4me3_PIC4peaks_intersect_00,human_prom1000100_H3K4me3_UNSTpeaks_intersect_00,human_prom1000100_H3K4me3_PIC4peaks_intersect_03,human_prom1000100_H3K4me3_UNSTpeaks_intersect_03,...,human_prom1M_H3K27ac_PIC4peaks_intersect_03,human_prom1M_H3K27ac_UNSTpeaks_intersect_03,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K27ac_PIC4peaks_intersect_00,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_00,mouse_prom1M_H3K27ac_PIC4peaks_intersect_03,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_03,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05
ENSG00000198888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000228253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000125740,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000139648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000185614,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000198400,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [44]:
human_mouse_ortho_1_1_chip_seq_cols=human_mouse_ortho_1_1_chip_seq_cols.loc[DE_genes_new_filtration]
human_mouse_ortho_1_1_chip_seq_cols

,human_prom1000100_H3K27ac_PIC4peaks_intersect_00,human_prom1000100_H3K27ac_UNSTpeaks_intersect_00,human_prom1000100_H3K27ac_PIC4peaks_intersect_03,human_prom1000100_H3K27ac_UNSTpeaks_intersect_03,human_prom1000100_H3K27ac_PIC4peaks_intersect_05,human_prom1000100_H3K27ac_UNSTpeaks_intersect_05,human_prom1000100_H3K4me3_PIC4peaks_intersect_00,human_prom1000100_H3K4me3_UNSTpeaks_intersect_00,human_prom1000100_H3K4me3_PIC4peaks_intersect_03,human_prom1000100_H3K4me3_UNSTpeaks_intersect_03,...,human_prom1M_H3K27ac_PIC4peaks_intersect_03,human_prom1M_H3K27ac_UNSTpeaks_intersect_03,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K27ac_PIC4peaks_intersect_00,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_00,mouse_prom1M_H3K27ac_PIC4peaks_intersect_03,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_03,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000198888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000228253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000169946,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000094661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000114770,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [45]:

atac_or_chip_Hagai_human_mouse=human_mouse_ortho_1_1_chip_seq_cols.copy()
atac_or_chip_Hagai_human_mouse

,human_prom1000100_H3K27ac_PIC4peaks_intersect_00,human_prom1000100_H3K27ac_UNSTpeaks_intersect_00,human_prom1000100_H3K27ac_PIC4peaks_intersect_03,human_prom1000100_H3K27ac_UNSTpeaks_intersect_03,human_prom1000100_H3K27ac_PIC4peaks_intersect_05,human_prom1000100_H3K27ac_UNSTpeaks_intersect_05,human_prom1000100_H3K4me3_PIC4peaks_intersect_00,human_prom1000100_H3K4me3_UNSTpeaks_intersect_00,human_prom1000100_H3K4me3_PIC4peaks_intersect_03,human_prom1000100_H3K4me3_UNSTpeaks_intersect_03,...,human_prom1M_H3K27ac_PIC4peaks_intersect_03,human_prom1M_H3K27ac_UNSTpeaks_intersect_03,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K27ac_PIC4peaks_intersect_00,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_00,mouse_prom1M_H3K27ac_PIC4peaks_intersect_03,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_03,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000198888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000198712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000228253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000169946,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000094661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
ENSG00000114770,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [46]:
mini_dfs=[]
for p in promoters:
    for intr in intersects:
        for h in histon:
            mini_df=atac_or_chip_Hagai_human_mouse[[i for i in atac_or_chip_Hagai_human_mouse.columns if p in i and f'intersect_{intr}' in i and h in i]]
            mini_dfs.append(mini_df)
        
mini_dfs[0]          


,human_prom1M_H3K27ac_PIC4peaks_intersect_00,human_prom1M_H3K27ac_UNSTpeaks_intersect_00,mouse_prom1M_H3K27ac_PIC4peaks_intersect_00,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_00
0,,,,
ENSG00000198888,0.0,0.0,0.0,0.0
ENSG00000198763,0.0,0.0,0.0,0.0
ENSG00000198804,0.0,0.0,0.0,0.0
ENSG00000198712,0.0,0.0,0.0,0.0
ENSG00000228253,0.0,0.0,0.0,0.0
...,...,...,...,...
ENSG00000169946,1.0,1.0,0.0,1.0
ENSG00000094661,1.0,1.0,0.0,1.0
ENSG00000114770,1.0,1.0,0.0,1.0


In [47]:
len(mini_dfs)

3

In [48]:
unst='UNST'
stim='PIC4'

res_dir_atac_1=r'{}\chip_seq_unst{}_stim{}\tables'.format(res_dir_top,unst,stim)
os.makedirs(res_dir_atac_1,exist_ok=True)

In [70]:
mini_stim_and_unst_group_genes_group_cond

,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05
0,,,,


In [71]:
df_for_group

,"[0, 0, 0, 0]","[0, 0, 0, 1]","[0, 0, 1, 0]","[0, 0, 1, 1]","[0, 1, 0, 0]","[0, 1, 0, 1]","[0, 1, 1, 0]","[0, 1, 1, 1]","[1, 0, 0, 0]","[1, 0, 0, 1]","[1, 0, 1, 0]","[1, 0, 1, 1]","[1, 1, 0, 0]","[1, 1, 0, 1]","[1, 1, 1, 0]","[1, 1, 1, 1]"
group_genes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.0,0.0
rest_genes,15.0,0.0,8.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,6.0,0.0,5.0,0.0,1353.0,0.0


In [69]:
mini_stim_and_unst_group_genes


,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05
0,,,,
ENSG00000170961,1,1,0,1
ENSG00000182307,1,1,0,1
ENSG00000131724,1,1,0,1
ENSG00000117143,1,1,0,1
ENSG00000166750,1,1,0,1
...,...,...,...,...
ENSG00000134775,1,1,0,1
ENSG00000137965,1,1,0,1
ENSG00000101017,1,1,0,1


In [49]:
total_dfs=[]
total_dfs_per=[]
df_fischer=pd.DataFrame()

for mini in mini_dfs:
    #df_total_for_case=pd.DataFrame(columns=[f'{peaks}-{h}, {p}, {intr}'])
    df_total_for_case_list=[]
    #index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
    #df_total_for_case_per=pd.DataFrame(columns=[f'{peaks}-{h}, {p}, {intr} percentage'])
    df_total_for_case_per_list=[]
    mini=mini.astype('int32')
    p=mini.columns[0].split('_')[1]
    #print(p)
    intr=mini.columns[0].split('peaks_')[1]
    #print(intr)
    h=mini.columns[0].split('_')[2]
    #print(h)
    #print(type(mini))
    mini_stim_and_unst=mini[[c for c in mini.columns if f'_{stim}peaks_' in c or f'_{unst}peaks_' in c]]
    rows_names=[]
    for k,v in groups_list.items():
        print(len(v))
        mini_stim_and_unst_group_genes=mini_stim_and_unst.loc[[i for i in v if i in mini_stim_and_unst.index]]
        mini_stim_and_unst_rest_genes=mini_stim_and_unst.loc[[i for i in mini_stim_and_unst.index if i not in v]]
        df_for_group=pd.DataFrame(index=['group_genes','rest_genes'])
        df_for_group_percentage=pd.DataFrame(index=['group_genes','rest_genes'])
        #print('newww')
        #print(mini_stim_and_unst_group_genes)
        #print(mini_stim_and_unst_rest_genes)
        
        print(f'for {peaks},{h} , {p} and {intr}:')
        print(f'for group: {k}:')
        print(f'total group genes: {len(mini_stim_and_unst_group_genes.index)}')
        print(f'total rest genes: {len(mini_stim_and_unst_rest_genes.index)}')
        print('\n')
        rows_names.append(f'{k}: total group genes: {len(mini_stim_and_unst_group_genes.index)}, total rest genes: {len(mini_stim_and_unst_rest_genes.index)}')
        for comb in combs_0_1:
            
            #print(f'for comb {comb}:[human_unst,human_stim,mouse_unst,mouse_stim]')
            mini_stim_and_unst_group_genes_group_cond=mini_stim_and_unst_group_genes[(mini_stim_and_unst_group_genes[f'human_{p}_{h}_{unst}peaks_{intr}']==comb[0]) & 
                                                                                        (mini_stim_and_unst_group_genes[f'human_{p}_{h}_{stim}peaks_{intr}']==comb[1]) & 
                                                                                        (mini_stim_and_unst_group_genes[f'mouse_{p}_{h}_{unst}peaks_{intr}']==comb[2]) & 
                                                                                        (mini_stim_and_unst_group_genes[f'mouse_{p}_{h}_{stim}peaks_{intr}']==comb[3]) ]
            
            mini_stim_and_unst_rest_genes_group_cond=mini_stim_and_unst_rest_genes[(mini_stim_and_unst_rest_genes[f'human_{p}_{h}_{unst}peaks_{intr}']==comb[0]) & 
                                                                                        (mini_stim_and_unst_rest_genes[f'human_{p}_{h}_{stim}peaks_{intr}']==comb[1]) & 
                                                                                        (mini_stim_and_unst_rest_genes[f'mouse_{p}_{h}_{unst}peaks_{intr}']==comb[2]) & 
                                                                                        (mini_stim_and_unst_rest_genes[f'mouse_{p}_{h}_{stim}peaks_{intr}']==comb[3]) ]
            df_for_group.at['group_genes',str(comb)]=len(mini_stim_and_unst_group_genes_group_cond.index)
            df_for_group.at['rest_genes',str(comb)]=len(mini_stim_and_unst_rest_genes_group_cond.index)
            
            df_for_group_percentage.at['group_genes',str(comb)]=len(mini_stim_and_unst_group_genes_group_cond.index)/len(mini_stim_and_unst_group_genes.index) *100
            df_for_group_percentage.at['rest_genes',str(comb)]=len(mini_stim_and_unst_rest_genes_group_cond.index)/len(mini_stim_and_unst_rest_genes.index) *100
        
            # make a ficher test
            # [[group-genes-peaks,rest-genes-peaks],[group-genes-NOpeaks,rest-genes-NOpeaks]
        #print(df_for_group)
        if k == f'{study}_{species[0]}_{species[1]}_{treats[1]}_high':
            comb_rel_col='[0, 0, 0, 1]'
        elif k == f'{study}_{species[0]}_{species[1]}_{treats[1]}_low':
            comb_rel_col='[0, 1, 0, 0]'
        elif k == f'{study}_{species[0]}_{species[1]}_{treats[0]}_high':
            comb_rel_col='[0, 1, 1, 1]'
        elif k == f'{study}_{species[0]}_{species[1]}_{treats[0]}_low':
            comb_rel_col='[1, 1, 0, 1]'
        
        print(k)
        print(comb_rel_col)
        group_genes_peaks=df_for_group.at['group_genes',comb_rel_col]
        print(group_genes_peaks)
        rest_genes_peaks=df_for_group.at['rest_genes',comb_rel_col]
        print(rest_genes_peaks)
        group_genes_NOpeaks=len(mini_stim_and_unst_group_genes.index)-group_genes_peaks
        print(group_genes_NOpeaks)
        rest_genes_NOpeaks=len(mini_stim_and_unst_rest_genes.index)-rest_genes_peaks
        print(rest_genes_NOpeaks)
        
        oddsratio, pvalue = fisher_exact([[group_genes_peaks, rest_genes_peaks], [group_genes_NOpeaks, rest_genes_NOpeaks]])
        print('for fisher!!!!')
        print([[group_genes_peaks, rest_genes_peaks], [group_genes_NOpeaks, rest_genes_NOpeaks]])
        df_fischer.at['{}_{}_{}_{}'.format(peaks,h,p,intr),k]=pvalue
            
        
        print('df_for_group')
        print(df_for_group)
        print('df_for_group_percentage')
        print(df_for_group_percentage)
        df_total_for_case_list.append(df_for_group)
        df_total_for_case_per_list.append(df_for_group_percentage)
        #df_total_for_case.at[f'{k}: \ngroup size: {len(mini_stim_and_unst_group_genes.index)} \nrest size:{len(mini_stim_and_unst_rest_genes.index)}',f'{peaks}-{h}, {p}, {intr}']=df_for_group
        #df_total_for_case_per.at[f'{k}: \ngroup size: {len(mini_stim_and_unst_group_genes.index)} \nrest size:{len(mini_stim_and_unst_rest_genes.index)}',f'{peaks}-{h}, {p}, {intr}']=df_for_group_percentage
        
    df_total_for_case=pd.concat(df_total_for_case_list,keys=rows_names)
    df_total_for_case_per=pd.concat(df_total_for_case_per_list,keys=rows_names)
    df_total_for_case=df_total_for_case.style.set_caption(f'{peaks}-{h}, {p}, {intr}')
    df_total_for_case_per=df_total_for_case_per.style.set_caption(f'{peaks}-{h}, {p}, {intr}')
    
    if ('2000500' in p) & ('27' in h) & ('00' in intr):
        useful_df=df_total_for_case
        useful_df_per=df_total_for_case_per
    
    total_dfs.append(df_total_for_case)
    total_dfs_per.append(df_total_for_case_per)
    print(intr)
    df_total_for_case.to_excel(r'{}\{}_{}_{}_{}.xlsx'.format(res_dir_atac_1,peaks,h,p,intr))
    print(type(df_total_for_case_per))

    path=r'{}\{}_{}_{}_{}_per.xlsx'.format(res_dir_atac_1,peaks,h,p,intr)
    #path=r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\my_project_from_22_11\more_oneTOone_analysis\chup_seq_atac_analysis_res\Hagai_homo_vs_mus_LF4_and_PIC4_groups_moderate\chip_seq_unstUNST_stimPIC4\tables\{}_{}_{}_{}_per.xlsx'.format(peaks,h,p,intr)
    #df_total_for_case_per.to_excel(path)
    
    #df_total_for_case_per.to_excel(res_dir_atac_1+'\\'+peaks+h+p+'_'+'intersect_00_'+'er.xlsx')
    
    #df_total_for_case_per.to_excel(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\my_project_from_22_11\more_oneTOone_analysis\chup_seq_atac_analysis_res\Hagai_homo_vs_mus_LF4_and_PIC4_groups_moderate\chip_seq_unstUNST_stimPIC4\tables\1_per.xlsx')
        

    
    
        #print(df_total_for_case)
        
        #print(df_total_for_case_per)
        #print(df_total_for_case)
        #print(df_total_for_case_per)
        
        #print('\n')
            
    

224
for chip_seq,H3K27ac , prom1M and intersect_00:
for group: Hagai_homo_mus_PIC4_high:
total group genes: 224
total rest genes: 1249


Hagai_homo_mus_PIC4_high
[0, 0, 0, 1]
0.0
0.0
224.0
1249.0
for fisher!!!!
[[0.0, 0.0], [224.0, 1249.0]]
df_for_group
             [0, 0, 0, 0]  [0, 0, 0, 1]  [0, 0, 1, 0]  [0, 0, 1, 1]  \
group_genes           1.0           0.0           1.0           0.0   
rest_genes           14.0           0.0           7.0           0.0   

             [0, 1, 0, 0]  [0, 1, 0, 1]  [0, 1, 1, 0]  [0, 1, 1, 1]  \
group_genes           0.0           0.0           0.0           0.0   
rest_genes            0.0           0.0           1.0           0.0   

             [1, 0, 0, 0]  [1, 0, 0, 1]  [1, 0, 1, 0]  [1, 0, 1, 1]  \
group_genes           0.0           0.0           1.0           0.0   
rest_genes            1.0           0.0           5.0           0.0   

             [1, 1, 0, 0]  [1, 1, 0, 1]  [1, 1, 1, 0]  [1, 1, 1, 1]  
group_genes           0.0         

<class 'pandas.io.formats.style.Styler'>
224
for chip_seq,H3K27ac , prom1M and intersect_05:
for group: Hagai_homo_mus_PIC4_high:
total group genes: 224
total rest genes: 1249


Hagai_homo_mus_PIC4_high
[0, 0, 0, 1]
0.0
0.0
224.0
1249.0
for fisher!!!!
[[0.0, 0.0], [224.0, 1249.0]]
df_for_group
             [0, 0, 0, 0]  [0, 0, 0, 1]  [0, 0, 1, 0]  [0, 0, 1, 1]  \
group_genes           1.0           0.0           1.0           0.0   
rest_genes           14.0           0.0           7.0           0.0   

             [0, 1, 0, 0]  [0, 1, 0, 1]  [0, 1, 1, 0]  [0, 1, 1, 1]  \
group_genes           0.0           0.0           0.0           0.0   
rest_genes            0.0           0.0           1.0           0.0   

             [1, 0, 0, 0]  [1, 0, 0, 1]  [1, 0, 1, 0]  [1, 0, 1, 1]  \
group_genes           0.0           0.0           1.0           0.0   
rest_genes            1.0           0.0           5.0           0.0   

             [1, 1, 0, 0]  [1, 1, 0, 1]  [1, 1, 1, 0]  [1, 1, 1

In [61]:
os.getcwd()

'C:\\Users\\TzachiHNB2\\Documents\\lilach_sc\\thesis\\chip_seq'

In [62]:
useful_df_per.to_excel(r'relevant_output\persentage_results_H3K27ac_prom2000500_inter00.xls')
useful_df.to_excel(r'relevant_output\numbers_results_H3K27ac_prom2000500_inter00.xls')

<ipython-input-62-b9254a4ca1ff>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  useful_df_per.to_excel(r'relevant_output\persentage_results_H3K27ac_prom2000500_inter00.xls')
<ipython-input-62-b9254a4ca1ff>:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, i

In [51]:
useful_df

NameError: name 'useful_df' is not defined

In [50]:
df_fischer

,Hagai_homo_mus_PIC4_high,Hagai_homo_mus_PIC4_low,Hagai_homo_mus_LF4_high,Hagai_homo_mus_LF4_low
chip_seq_H3K27ac_prom1M_intersect_00,1.0,1.0,1.0,1.0
chip_seq_H3K27ac_prom1M_intersect_03,1.0,1.0,1.0,1.0
chip_seq_H3K27ac_prom1M_intersect_05,1.0,1.0,1.0,1.0


In [53]:
df_fischer.to_csv(r'{}\chip_seq_unst{}_stim{}\fischer_pvals.csv'.format(res_dir_top,unst,stim))
df_fischer

,Hagai_homo_mus_PIC4_high,Hagai_homo_mus_PIC4_low,Hagai_homo_mus_LF4_high,Hagai_homo_mus_LF4_low
chip_seq_H3K27ac_prom1M_intersect_00,1.0,1.0,1.0,1.0
chip_seq_H3K27ac_prom1M_intersect_03,1.0,1.0,1.0,1.0
chip_seq_H3K27ac_prom1M_intersect_05,1.0,1.0,1.0,1.0
